In [1]:
# http://files.grouplens.org/datasets/movielens/ml-20m.zip
import pandas as pd
from sklearn import model_selection
import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader


df = pd.read_csv('./ml-20m/ratings.csv')
X = df[['userId', 'movieId']].values
Y = df[['rating']].values
train_X, test_X, train_Y, test_Y = model_selection.train_test_split(X, Y, test_size=0.1)
train_dataset = TensorDataset(torch.LongTensor(train_X), torch.FloatTensor(train_Y))
test_dataset = TensorDataset(torch.LongTensor(test_X), torch.FloatTensor(test_Y))
train_loader = DataLoader(train_dataset, batch_size=1024, num_workers=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1024, num_workers=4)

In [2]:
from torch import nn


class MatrixFactorization(nn.Module):
    def __init__(self, max_user, max_item, k=20):
        super().__init__()
        self.max_user = max_user
        self.max_item = max_item
        self.user_emb = nn.Embedding(max_user, k, 0)
        self.item_emb = nn.Embedding(max_item, k, 0)
        
    def forward(self, x):
        user_idx = x[:, 0]
        item_idx = x[:, 1]
        user_feature = self.user_emb(user_idx)
        item_feature = self.item_emb(item_idx)
        out = torch.sum(user_feature*item_feature, 1)
        out = nn.functional.sigmoid(out) * 5
        return out

In [3]:
max_user, max_item = X.max(0)
max_user = int(max_user)
max_item = int(max_item)
net = MatrixFactorization(max_user+1, max_item+1)

In [4]:
from torch.autograd import Variable as V
from statistics import mean


def mae(x, y):
    return (x - y).abs().mean()

def eval_net(net, loader, score_fn=mae):
    ys = []
    ypreds = []
    for x, y in loader:
        x = V(x)
        ys.append(y)
        ypred = net(x).data
        ypreds.append(ypred)
    score = score_fn(torch.cat(ys).squeeze(), torch.cat(ypreds))
    return score

In [5]:
import resource
from torch import optim


rlimit = resource.getrlimit(resource.RLIMIT_NOFILE)
resource.setrlimit(resource.RLIMIT_NOFILE, (2048, rlimit[1]))

opt = optim.Adam(net.parameters(), lr=0.01)
loss_f = nn.MSELoss()
for epoch in range(5):
    loss_log = []
    for x, y in train_loader:
        x = V(x)
        y = V(y)
        o = net(x)
        loss = loss_f(o, y)
        net.zero_grad()
        loss.backward()
        opt.step()
        loss_log.append(loss.data[0])
    test_score = eval_net(net, test_loader)
    print(epoch, mean(loss_log), test_score, flush=True)

0 1.6108203682112026 0.7333635936163142
1 0.8801158481751213 0.7086873197575217
2 0.8364924134583703 0.7018883533381426
3 0.81532753906435 0.697225111517906
4 0.8023938441467784 0.695905596217421


In [6]:
query = (1, 10)
query = torch.LongTensor(query).view(1, -1)
net(V(query))

Variable containing:
 3.2269
[torch.FloatTensor of size 1]

In [7]:
query = torch.stack([
    torch.zeros(max_item).fill_(1),
    torch.arange(1, max_item+1)
], 1).long()
scores, indices = torch.topk(net(V(query)), 5)
print(scores, indices)

Variable containing:
 5.0000
 5.0000
 5.0000
 5.0000
 5.0000
[torch.FloatTensor of size 5]
 Variable containing:
 41322
  7652
 77913
 84616
 70887
[torch.LongTensor of size 5]

